In [0]:
# ——————————————————————————————
#  ENVIRONMENT CONFIGURATION
# ——————————————————————————————
dbutils.widgets.text("pipeline.env", "dev")
env = dbutils.widgets.get("pipeline.env")
catalog = "book_rec_catalog"
silver_schema = f"{env}_silver"
gold_schema = f"{env}_gold"

spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {gold_schema}")

print(f"Catalog: {catalog}")
print(f"Silver Schema: {silver_schema}")
print(f"Gold Schema: {gold_schema}")

Top 10 books

In [0]:
# I want to create a view that will show top 10 books by popularity score
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.v_top_10_books_ever AS
SELECT 
    b.`Book-Title`,
    b.`Book-Author`,
    b.`Publisher`,
    b.`Year-Of-Publication`,
    COUNT(*)                                      AS total_ratings,
    AVG(CAST(r.`Book-Rating` AS DOUBLE))          AS avg_rating,
    SUM(CAST(r.`Book-Rating` AS DOUBLE))          AS popularity_score
FROM {catalog}.{silver_schema}.books_silver_batch b
JOIN {catalog}.{silver_schema}.ratings_silver_batch r 
  ON b.ISBN = r.ISBN
WHERE r.`Book-Rating` > 0
GROUP BY b.`Book-Title`, b.`Book-Author`, b.`Publisher`, b.`Year-Of-Publication`
HAVING COUNT(*) >= 5
ORDER BY popularity_score DESC, avg_rating DESC, total_ratings DESC, b.`Book-Title`
LIMIT 10;
""")

print("View v_top_10_books created successfully!")

Top 10 authors

In [0]:
# I want to create a view that will show top 10 authors by popularity score and average rating
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.v_top_10_authors AS
WITH base AS (
  SELECT
    b.`Book-Author` AS author,
    b.ISBN          AS isbn,
    r.`User-ID`     AS user_id,
    CAST(r.`Book-Rating` AS DOUBLE) AS rating
  FROM {catalog}.{silver_schema}.books_silver_batch b
  JOIN {catalog}.{silver_schema}.ratings_silver_batch r
    ON b.ISBN = r.ISBN
  WHERE r.`Book-Rating` > 0
),
dedup AS (  -- If same user did multiple ratings, take average
  SELECT author, isbn, user_id, AVG(rating) AS rating
  FROM base
  GROUP BY author, isbn, user_id
)
SELECT
  author,
  COUNT(*)                    AS total_ratings,
  COUNT(DISTINCT user_id)     AS users_count,
  COUNT(DISTINCT isbn)        AS books_count,
  AVG(rating)                 AS avg_rating,
  SUM(rating)                 AS popularity_score
FROM dedup
GROUP BY author
HAVING COUNT(*) >= 10 AND COUNT(DISTINCT user_id) >= 10 -- Minimum 10 ratings and 10 users to be fair to the author
ORDER BY avg_rating DESC, total_ratings DESC, books_count DESC, author
LIMIT 10;

""")

Top authors by year

In [0]:
# I want to create a view that will show top authors by year of publication
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.v_authors_top10_by_year AS
WITH base AS (
  SELECT
    CAST(b.`Year-Of-Publication` AS INT) AS year_pub,
    TRIM(b.`Book-Author`)                AS author,
    b.ISBN                               AS isbn,
    r.`User-ID`                          AS user_id,
    CAST(r.`Book-Rating` AS DOUBLE)      AS rating
  FROM {catalog}.{silver_schema}.books_silver_batch b
  JOIN {catalog}.{silver_schema}.ratings_silver_batch r ON b.ISBN = r.ISBN
  WHERE r.`Book-Rating` > 0
    AND CAST(b.`Year-Of-Publication` AS INT) BETWEEN 1990 AND 2024
),
dedup AS (  -- 1 user × 1 book = 1 vote
  SELECT year_pub, author, isbn, user_id, AVG(rating) AS rating
  FROM base
  GROUP BY year_pub, author, isbn, user_id
),
agg AS (
  SELECT
    year_pub,
    author,
    COUNT(*)                AS total_ratings,
    COUNT(DISTINCT user_id) AS users_count,
    COUNT(DISTINCT isbn)    AS books_count,
    AVG(rating)             AS avg_rating
  FROM dedup
  GROUP BY year_pub, author
  HAVING COUNT(*) >= 10 AND users_count >= 10 AND books_count >= 2
),
ranked AS (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY year_pub
           ORDER BY avg_rating DESC, total_ratings DESC, books_count DESC, author
         ) AS rn
  FROM agg
)
SELECT year_pub AS `Year-Of-Publication`,
       author   AS `Book-Author`,
       total_ratings, users_count, books_count, avg_rating
FROM ranked
WHERE rn <= 10
ORDER BY year_pub DESC, rn ASC;

""")

print("View v_top_authors_by_year created successfully!")